# Criando pequenos inputs

In [ ]:
import pandas as pd

df = pd.read_csv("scraper/input/hotels_24_merged.csv")

In [ ]:
df2 = pd.read_csv("scraper/input/hotels_2024.csv")
df2.shape

(1356, 9)

In [ ]:
df2[~df2.google_id.isin(df.google_id)].shape

(0, 9)

In [ ]:
df[~df.google_id.isin(df2.google_id)]

,done,dt_selection,name,n_reviews,sort_by,hl,n_reviews_max,google_id,url
1270,0,2023,Hotel Costa Norte - Ingleses,-1,most_relevant,pt-br,1024,0x0:0xecdacd0a9c8f6360,https://www.google.com/maps/place/Hotel+Costa+...
1274,0,2023,Céu Palmas Hotel,-1,most_relevant,pt-br,1057,0x0:0x75bff227c9aad85a,https://www.google.com/maps/place/C%C3%A9u+Pal...
1282,0,2023,Aroso Paço Hotel,-1,most_relevant,pt-br,1105,0x0:0xfebbe61e81d2bc2d,https://www.google.com/maps/place/Aroso+Pa%C3%...
1286,0,2023,Guarany Hotel Express,-1,most_relevant,pt-br,1127,0x0:0xb47c04c354412853,https://www.google.com/maps/place/Guarany+Hote...
1287,0,2023,Boa Vista Eco Hotel,-1,most_relevant,pt-br,1132,0x0:0x44641140201ca772,https://www.google.com/maps/place/Boa+Vista+Ec...
1304,0,2023,Hotel Express,-1,most_relevant,pt-br,1233,0x0:0x973b70772551b05c,https://www.google.com/maps/place/Hotel+Expres...
1308,0,2023,Grande Hotel Ipatinga,-1,most_relevant,pt-br,1269,0x0:0x7e6468726a517be6,https://www.google.com/maps/place/Grande+Hotel...
1309,0,2023,Posseidon Hotel,-1,most_relevant,pt-br,1291,0x0:0x5c40013cb556cc81,https://www.google.com/maps/place/Posseidon+Ho...
1317,0,2023,Atrium Quinta de Pedras,-1,most_relevant,pt-br,1375,0x0:0x66dc9a457771e065,https://www.google.com/maps/place/Atrium+Quint...
1320,0,2023,Atlântico Center,-1,most_relevant,pt-br,1382,0x0:0xcb0d68682790adde,https://www.google.com/maps/place/Atl%C3%A2nti...


In [ ]:
df2.iloc[0]

done                                                 0
name                                        Hotel Sara
n_reviews                                           -1
sort_by                                  most_relevant
hl                                               pt-br
url              0x91f155649845b03b:0xe6966179c101261f
google_id        0x91f155649845b03b:0xe6966179c101261f
n_reviews_max                                        5
dt_selection                                      2024
Name: 0, dtype: object

In [ ]:
df.shape

(1406, 9)

In [ ]:
df["sort_by"] = "newest"

In [ ]:
df[df.n_reviews_max.cumsum() < 10_000].shape

(426, 9)

In [ ]:
cumsum = df.n_reviews_max.cumsum()
cumsum

0            5
1           10
2           15
3           20
4           25
         ...  
1401    604404
1402    622477
1403    643219
1404    664814
1405    688547
Name: n_reviews_max, Length: 1406, dtype: int64

In [ ]:
df["batch"] = -1

In [ ]:
dfs = []
step = 20_000
path = "./scraper/input/2025/"
for i in range(40):
    f = (cumsum > step*i) & (cumsum <= step*(i+1))
    df.loc[f, "batch"] = i
    dfi = df.loc[f]
    print(i, dfi.shape)
    dfs.append(dfi)
    dfi.to_csv(path+"hotels_batch_"+str(i).rjust(2,"0")+".csv", index=False)

0 (576, 10)
1 (162, 10)
2 (108, 10)
3 (83, 10)
4 (69, 10)
5 (56, 10)
6 (46, 10)
7 (41, 10)
8 (35, 10)
9 (30, 10)
10 (26, 10)
11 (23, 10)
12 (21, 10)
13 (18, 10)
14 (16, 10)
15 (14, 10)
16 (13, 10)
17 (11, 10)
18 (10, 10)
19 (7, 10)
20 (7, 10)
21 (6, 10)
22 (4, 10)
23 (4, 10)
24 (3, 10)
25 (3, 10)
26 (3, 10)
27 (2, 10)
28 (2, 10)
29 (2, 10)
30 (1, 10)
31 (1, 10)
32 (1, 10)
33 (1, 10)
34 (1, 10)
35 (0, 10)
36 (0, 10)
37 (0, 10)
38 (0, 10)
39 (0, 10)


In [ ]:
df.to_csv("./scraper/input/hotels_2025.csv", index=False)

# Normalizando e transformando índices

In [1]:
from uuid import uuid4
import faiss
import pandas as pd
import numpy as np
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.globals import set_debug
import asyncio
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import logging
from datetime import datetime
import time
import json
from tqdm import tqdm
import os
import re

from llms.utils import timeit
from llms.models import load_model, load_embedding, query_model, query_model_async

# logging.basicConfig(level=logging.DEBUG)
# set_debug(True)

QUERY_PREFIX_E5_INSTRUCT = "Instruct: Given a Portuguese question, retrieve relevant hotel reviews that best answer the question. \nQuery: "
DOC_PREFIX = ""
QUERY_PREFIX = ""

CREATE_INDEX = True
PATH_INDEX = "data/faiss_index_gte_v5"
INDEX_BATCH_SIZE = 10_000
N_SAMPLES = None
N_RESPONSES = 5
FETCH_K = 1000
MAX_VECTOR_STORES = 1000
# PATH_DATA = "data/df_prep_2024-12-09_08-23-45_627733.pq"
PATH_DATA = "data/df_index_2025_v1.pq"
EMBEDDING_MODEL = "gte"

In [2]:
folders = os.listdir(PATH_INDEX)
f = folders[0]

In [6]:
embeddings, _ = load_embedding("gte", task_type="retrieval_document")

load_embedding...
model_alias='gte' model_name='Alibaba-NLP/gte-multilingual-base' task_type='retrieval_document'
Loading HuggingFaceEmbeddings model...


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.bias', 'classifier.weight'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\Bruno\miniconda3\envs\cu124\Lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


embeddings=HuggingFaceEmbeddings(model_name='Alibaba-NLP/gte-multilingual-base', cache_folder=None, model_kwargs={'trust_remote_code': True}, encode_kwargs={}, multi_process=False, show_progress=False)


In [7]:
embeddings.embed_query("textoooooo bem quadrado bem normalizado").__len__()

768

In [30]:
vector_store_l2 = FAISS.load_local(
    f"{PATH_INDEX}/{f}",
    embeddings,
    allow_dangerous_deserialization=True,
)

index: faiss.Index = vector_store_l2.index
vectors = index.reconstruct_n(0, index.ntotal)

In [40]:
index_ip = faiss.IndexFlatIP(index.d)
index_ip.add(vectors)

In [41]:
vector_store_ip = FAISS(
    embedding_function=embeddings,
    index=index_ip,
    docstore=InMemoryDocstore(vector_store_l2.docstore._dict),
    index_to_docstore_id=vector_store_l2.index_to_docstore_id,
)

In [ ]:
(vector_store_ip.index.reconstruct_n(0, 1)[0] == vector_store_l2.index.reconstruct_n(0, 1)[0]).all()

True

In [50]:
assert vector_store_ip.index.d == vector_store_l2.index.d
assert vector_store_ip.index.ntotal == vector_store_l2.index.ntotal
assert vector_store_ip.docstore._dict == vector_store_l2.docstore._dict
assert vector_store_ip.index_to_docstore_id == vector_store_l2.index_to_docstore_id
assert (vector_store_ip.index.reconstruct_n(0, 1)[0] == vector_store_l2.index.reconstruct_n(0, 1)[0]).all()

In [52]:
np.linalg.norm(vector_store_ip.index.reconstruct_n(0, 1)[0]) == 1

True

In [24]:
# get all indexes of index
vectors = index.reconstruct_n(0, index.ntotal)

In [ ]:
index_to_docstore_id = vector_store_l2.index_to_docstore_id

{0: '3b253b8c-9ecb-4a47-a29b-597057358e77',
 1: '497d3ac4-fb44-4d48-a507-c280c3aa16a2',
 2: '81fc7939-f450-48d5-8407-ece5fea5e6fd',
 3: '17e154ec-82d5-48ab-b922-e0660575ce4b',
 4: '83ec37ac-b454-4675-98f6-355e765b0dba',
 5: '081cd0af-2dc3-4ec9-9c83-0f4d85ac9dba',
 6: '10c356e0-6839-4760-9349-b8db092ac158',
 7: 'b42a88cb-b8ad-4314-80a7-b1560f6c1145',
 8: '84c86667-c075-4d7a-8b05-181c6886703a',
 9: 'c579521e-258c-4b15-a653-2f1b11dfcd63',
 10: 'e0081be3-2686-4f90-8375-5a7e958648c5',
 11: '36499b51-52e1-4e62-86ee-8bcfa8320735',
 12: 'f8dc59ef-7798-498d-b7ee-436309c491c9',
 13: '65ffecca-1fd5-46a2-b921-10c402691a53',
 14: 'd59fb3fb-1772-4a47-a6ec-aa6abe92da79',
 15: 'bdd6d169-85c0-4f51-9e9d-fc93982d2db7',
 16: '1aa9e790-d821-416d-9770-ef51aeda3762',
 17: '1fe0f874-eebf-444b-b9e9-1954e9ea6e3f',
 18: '596f6d82-73dc-4371-b5e8-9f7d0ca325ae',
 19: '9ec25bcd-6f23-431c-a129-bd9888bcdbea',
 20: '4d20ebdd-5a82-408c-b770-52a70a4bb6ae',
 21: '72fd8d02-1e13-40ec-91ae-767de9518c35',
 22: 'b52e3b29-ec15-

In [19]:
from langchain_community.vectorstores import InMemoryVectorStore
metadata = vector_store_l2.docstore._dict

In [22]:
metadata['3b253b8c-9ecb-4a47-a29b-597057358e77'].page_content

'Excelente custo benefício. Sempre me hospedo lá quando vou pra Serra Negra. Primeira coisa que tem que ficar claro é que é uma hospedagem e não um hotel. O proprietário é muito gente boa e presta assistência dentro do possível. Localização é excelente, na praça dos barzinhos e do lado da Padaria Serrana. Se gosta de agito, é o local perfeito. Se busca silêncio, melhor buscar outra opção mais isolada. Os quartos são grandes. Os preços são bem compatíveis.'

In [ ]:
vector_store_l2.add_embeddings(vectors, )

AttributeError: 'FAISS' object has no attribute 'get_metadata'

In [21]:
pd.Series(vectors[0]).describe()


count    768.000000
mean       0.001230
std        0.036087
min       -0.103693
25%       -0.023479
50%        0.000864
75%        0.022709
max        0.170489
dtype: float64

In [38]:
np.linalg.norm(vectors[0])

1.0

In [34]:
def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
vectors_normalized = normalize(vectors)

In [36]:
np.linalg.norm(vectors_normalized[0])

1.0

In [27]:
pd.Series(vectors_normalized[0]).describe()

count    768.000000
mean       0.001230
std        0.036087
min       -0.103693
25%       -0.023479
50%        0.000864
75%        0.022709
max        0.170489
dtype: float64

In [ ]:
def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)


    # NORMALIZE ALL EMBEDDINGS OF vector_store_l2
    for doc in vector_store_l2.docstore.documents:
    vector.page_content = normalize(doc.page_content)

# Sumarização

In [1]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
import traceback
import json

from llms.models import (
    load_model,
    query_model,
    models_text,
    models_embedding,
)
from llms.rag import load_rag, query_index, query_make_filter, load_data

In [4]:
df = load_data()

Loading data...
(222923, 22)
(222923, 22)
load_data took: 0.6748313903808594


In [9]:
df.groupby("nome").agg(
    count_=("text", "count"),
).sort_values("count_")

,count_
nome,
Hotel Brito,1
Padaria do Radiola (Radiopão) e Hotel São José,1
Vitória da Serra Pousada e Eventos,1
Pousada Ipe Amarelo,1
Pousada Jacuí,1
...,...
Copacabana Palace,6025
Hotel Villa Lobos Spa Romantik,6672
Costao do Santinho Resort,8160


In [14]:
df_hotel = df.query("nome=='Copacabana Palace'")

In [ ]:
from langchain_core.documents import Document

docs = []
for i, (id, row) in enumerate(df_hotel.iterrows()):
    metadata = row.to_dict()
    text = metadata.pop("text")
    doc = Document(page_content=text, metadata=metadata)
    docs.append(doc)


In [58]:
from llms.prompts import format_context

def make_context_summary_full(docs):
    meta = docs[0].__dict__["metadata"]
    context = (
        f"Hotel: {meta['nome']}, {int(meta['estrelas'])} Estrelas.\n"
        f"Região:{meta['regiao']}; Estado:{meta['estado']}; Cidade:{meta['cidade']}\n"
        f"Tipo:{meta['subcategoria']}; Classificação:{meta['classificacao_geral']}; Quantidade Avaliações:{int(meta['quantidade_avaliacoes'])}\n"
        "\n"
    )

    for i in range(len(docs)):
        context_i = format_context(i, docs[i], include_hotel_context=False)
        context += context_i

    return context

context = make_context_summary_full(docs[:100])
print(context)

Hotel: Copacabana Palace, 5 Estrelas.
Região:SUDESTE; Estado:RIO DE JANEIRO; Cidade:None
Tipo:None; Classificação:4.8; Quantidade Avaliações:37383

 - Avaliação 1
Nota:5; Curtidas:0; Usuário é guia local; Nota Quartos:5; Nota Localização:5; Nota Serviço:5
Avaliação: Lugar de muita história de fácil acesso. Amei estar aí. O coffee-break é simplicidade maravilhoso.

 - Avaliação 2
Nota:5; Curtidas:0; Usuário é guia local
Avaliação: O Freddie Mercury Swite, o hotel escolhido por tal estrela por um motivo  . O melhor.

 - Avaliação 3
Nota:5; Curtidas:0; Usuário é guia local
Avaliação: é um lindo hotel. Gostei muito pelo atendimento e hospedagem

 - Avaliação 4
Nota:5; Curtidas:0; Usuário é guia local; Nota Quartos:5; Nota Localização:5; Nota Serviço:5
Avaliação: Ótimo lugar e muito bem localizado, adorei o atendimento e fui muito bem recebido.

 - Avaliação 5
Nota:5; Curtidas:0; Usuário é guia local; Nota Quartos:5; Nota Localização:5; Nota Serviço:5
Avaliação: O Copacabana Palace superou 

In [59]:
from llms.prompts import PROMPT_SUMMARY, topics

def make_query_summary_full(topics):
    query = "Faça um resumo do hotel com base nos seguintes tópicos:\n"
    for i, t in enumerate(topics):
        query += f"{i+1}. {t}\n"
    query += "Para cada tópico, escreva um parágrafo com os principais aspectos positivos e negativos do hotel com base em suas avaliações."
    return query

query = make_query_summary_full(topics)
print(query)

Faça um resumo do hotel com base nos seguintes tópicos:
1. Infraestrutura e Acomodações – Conforto, limpeza, tecnologia, lazer, estacionamento.
2. Atendimento e Serviço – Cordialidade, eficiência, limpeza, concierge, check-in ágil.
3. Localização e Acessibilidade – Proximidade, transporte, segurança, acessibilidade.
4. Alimentação e Bebidas – Café da manhã, restaurante, serviço de quarto, qualidade.
5. Experiência e Entretenimento – Lazer, eventos, recreação, passeios, parcerias.
6. Custo-benefício e Políticas – Preço justo, flexibilidade, transparência, fidelidade.
Para cada tópico, escreva um parágrafo com os principais aspectos positivos e negativos do hotel com base em suas avaliações.


In [60]:
prompt_final = PROMPT_SUMMARY.format(context=context, query=query)
print(prompt_final)


Você é um assistente de sumarização de hotéis em português.
Utilize os seguintes trechos de CONTEXTO recuperado de avaliações de hotéis para escrever o RESUMO.
Siga as INSTRUÇÕES do usuário para escrever um resumo detalhado do que se pede.

CONTEXTO:
Hotel: Copacabana Palace, 5 Estrelas.
Região:SUDESTE; Estado:RIO DE JANEIRO; Cidade:None
Tipo:None; Classificação:4.8; Quantidade Avaliações:37383

 - Avaliação 1
Nota:5; Curtidas:0; Usuário é guia local; Nota Quartos:5; Nota Localização:5; Nota Serviço:5
Avaliação: Lugar de muita história de fácil acesso. Amei estar aí. O coffee-break é simplicidade maravilhoso.

 - Avaliação 2
Nota:5; Curtidas:0; Usuário é guia local
Avaliação: O Freddie Mercury Swite, o hotel escolhido por tal estrela por um motivo  . O melhor.

 - Avaliação 3
Nota:5; Curtidas:0; Usuário é guia local
Avaliação: é um lindo hotel. Gostei muito pelo atendimento e hospedagem

 - Avaliação 4
Nota:5; Curtidas:0; Usuário é guia local; Nota Quartos:5; Nota Localização:5; Nota 

In [54]:
llm, _, _ = load_model("gemini-2.0-flash")

load_model...
model_alias='gemini-2.0-flash' model_name='gemini-2.0-flash' max_new_tokens=3000
Loading ChatGoogleGenerativeAI model...
llm=<llms.models.GeminiHuggingFacePipeline object at 0x000002473C74DC10>


In [61]:
from llms.models import query_model

response = query_model(llm, prompt_final)

query_model...
len(prompt)=29578
num_input_tokens=4358
len(response)=3121
response='## Resumo do Copacabana Palace com base nas avaliações:\n\n**1. Infraestrutura e Acomodações:**\n\nO Copacabana Palace é amplamente elogiado por sua infraestrutura luxuosa e acomodações confortáveis. Os quartos são descritos como aconchegantes, espaçosos, bem decorados, limpos e equipados com detalhes que garantem uma estadia agradável. Muitos destacam a vista deslumbrante para o mar ou para os jardins. A arquitetura clássica do hotel, com toques modernos, impressiona os hóspedes, proporcionando um ambiente elegante e sofisticado. No entanto, algumas avaliações mencionam que não é possível entrar no hotel sem reserva.\n\n**2. Atendimento e Serviço:**\n\nO atendimento é um dos pontos mais fortes do Copacabana Palace, com muitos hóspedes elogiando a cordialidade, educação e prestatividade dos funcionários. A equipe é descrita como atenciosa, eficiente e sempre disposta a ajudar, proporcionando um serviço 

In [62]:
print(response)

## Resumo do Copacabana Palace com base nas avaliações:

**1. Infraestrutura e Acomodações:**

O Copacabana Palace é amplamente elogiado por sua infraestrutura luxuosa e acomodações confortáveis. Os quartos são descritos como aconchegantes, espaçosos, bem decorados, limpos e equipados com detalhes que garantem uma estadia agradável. Muitos destacam a vista deslumbrante para o mar ou para os jardins. A arquitetura clássica do hotel, com toques modernos, impressiona os hóspedes, proporcionando um ambiente elegante e sofisticado. No entanto, algumas avaliações mencionam que não é possível entrar no hotel sem reserva.

**2. Atendimento e Serviço:**

O atendimento é um dos pontos mais fortes do Copacabana Palace, com muitos hóspedes elogiando a cordialidade, educação e prestatividade dos funcionários. A equipe é descrita como atenciosa, eficiente e sempre disposta a ajudar, proporcionando um serviço impecável. O serviço de quarto é frequentemente mencionado como magnífico e rápido. Alguns h

In [ ]:
vector_store, _, _ = load_rag("google-ip")
llm, _, _ = load_model("gemini-2.0-flash")